# 1. Подготовка

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
import re

from tqdm import notebook
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer 

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Прочитаем файл данных

In [2]:
url = "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1LJuNfEyryR-pJVJX4H-2-MRJAKyNUxke"

try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv(url)

print('Процентное содержание токсичных комментариев: {:.2%}'.format(data.toxic.mean()))
display(data.head())

Процентное содержание токсичных комментариев: 10.17%


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Создадим функции, которые будут очищать текст от символов, цифр и т.д., оставляя только английские буквы, а также лемматизировать текст.

In [3]:
def clear_text(text):
    """
    Функция оставляет в тексте только буквы английского алфавита.
    """
    text.replace('\n', '')
    only_letters = re.sub(r'[^a-zA-Z ]', ' ', text)
    return ' '.join(only_letters.split())

def lemmatize(text):
    """
    Функция лемматизирует текст.
    """    
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemm_text

Загрузим списки слов для лемматизации и стоп-слова, необходимые для преобразования текстов.

In [4]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stopwords = set(nltk_stopwords.words('english'))

# инициализируем объект WordNetLemmatizer для лемматизации
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Создадим новый столбец `lemm_text`, в который добавим тексты после очистки и лемматизации.

In [5]:
notebook.tqdm.pandas()

data['lemm_text'] = data['text'].progress_apply(lambda x: clear_text(lemmatize(x)))

Разобьем выборку на обучающую и тестовую в отношении 80:20.

In [6]:
train, test = train_test_split(data, test_size=0.2, random_state=404)

print('Размер обучающей выборки:', train.shape)
print('Размер тестовой выборки:', test.shape)

Размер обучающей выборки: (127656, 3)
Размер тестовой выборки: (31915, 3)


Из лемматизированного текста создадим признаки с помощью преобразования TF-IDF. Сначала преобразуем обучающую выборку.

In [7]:
train_corpus = train['lemm_text'].values.astype('U')

count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
train_tf_idf = count_tf_idf.fit_transform(train_corpus)

print('Размер обучающей выборки после преобразования TF-IDF:', train_tf_idf.shape)

Размер обучающей выборки после преобразования TF-IDF: (127656, 144651)


Теперь преобразуем тестовую выборку.

In [8]:
test_corpus = test['lemm_text'].values.astype('U')

test_tf_idf = count_tf_idf.transform(test_corpus)
print('Размер тестовой выборки после преобразования TF-IDF:', test_tf_idf.shape)

Размер тестовой выборки после преобразования TF-IDF: (31915, 144651)


Признаки готовы для передачи в модель для обучения.

# 2. Обучение

## 2.1. Логистическая регрессия

Обучим модель логистической регрессии.

In [9]:
%%time

model_lr = LogisticRegression(solver='lbfgs', max_iter=200)
model_lr.fit(train_tf_idf, train['toxic'])
y_pred = model_lr.predict(test_tf_idf)
y_test = test['toxic']

print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))
print('F1 score: {:.3f}'.format(f1_score(y_test, y_pred)))

Accuracy: 0.955
F1 score: 0.729
CPU times: user 9.03 s, sys: 6.22 s, total: 15.3 s
Wall time: 7.8 s


## 2.2. Линейный SVC

Обучим модель линейного SVC.

In [10]:
%%time

model_svc = LinearSVC(penalty = 'l1', dual=False)
model_svc.fit(train_tf_idf, train['toxic'])
y_pred = model_svc.predict(test_tf_idf)
y_test = test['toxic']

print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))
print('F1 score: {:.3f}'.format(f1_score(y_test, y_pred)))

Accuracy: 0.960
F1 score: 0.781
CPU times: user 2.98 s, sys: 97.8 ms, total: 3.08 s
Wall time: 3.02 s


## 2.3. Дерево решений

Обучим модель дерева решений.

In [ ]:
%%time

model_dt = DecisionTreeClassifier(random_state=404)
model_dt.fit(train_tf_idf, train['toxic'])
y_pred = model_dt.predict(test_tf_idf)
y_test = test['toxic']

print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))
print('F1 score: {:.3f}'.format(f1_score(y_test, y_pred)))

# Accuracy: 0.941
# F1 score: 0.705
# CPU times: user 5min 40s, sys: 33.2 ms, total: 5min 40s
# Wall time: 5min 41s

## 2.4. Случайный лес

Обучим модель случайного леса

In [ ]:
%%time

model_rf = RandomForestClassifier(random_state=404)
model_rf.fit(train_tf_idf, train['toxic'])
y_pred = model_rf.predict(test_tf_idf)
y_test = test['toxic']

print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))
print('F1 score: {:.3f}'.format(f1_score(y_test, y_pred)))

# Accuracy: 0.952
# F1 score: 0.701
# CPU times: user 12min 28s, sys: 588 ms, total: 12min 28s
# Wall time: 12min 29s

## 2.5. Градиентный бустинг от LGBM

Обучим модель градиентного бустинга.

In [11]:
%%time

model_lgbm = LGBMClassifier(learning_rate=0.5)
model_lgbm.fit(train_tf_idf, train['toxic'])
y_pred = model_lgbm.predict(test_tf_idf)
y_test = test['toxic']

print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))
print('F1 score: {:.3f}'.format(f1_score(y_test, y_pred)))

Accuracy: 0.953
F1 score: 0.749
CPU times: user 1min 58s, sys: 415 ms, total: 1min 59s
Wall time: 1min 1s


# 3. Выводы

Лучший результат по метрике F1 при обучении показала модель LinearSVC - **F1 = 0.781**, при этом она же оказалась самой быстрой - обучение с предсказанием заняло менее 5 секунд. С остальными моделями не удалось достичь требуемого качества модели по метрике F1.

# 4. Использование предобученной модели BERT

Импортируем необходимые библиотеки, проверим наличие графического процессора.

In [12]:
pip install transformers

     |████████████████████████████████| 1.4MB 5.8MB/s 
     |████████████████████████████████| 2.9MB 32.5MB/s 
     |████████████████████████████████| 890kB 36.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0707e1c1b9a622fbe10dba3e785dbbbe86b242ecfe2f71109177432dfeef2013
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [13]:
import torch
import transformers

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using Tesla P100-PCIE-16GB GPUs


Выполним очистку текста от лишних символов.

In [15]:
notebook.tqdm.pandas()

data['text_clear'] = data['text'].progress_apply(lambda x: clear_text(x))

Выполним инициализацию объектов `BertTokenizer`, `BertConfig`, `BertModel`.

In [16]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
config = transformers.BertConfig()
model = transformers.BertModel.from_pretrained('bert-base-uncased', config=config)
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

Выполним токенизацию очищенного текста с помощью `BertTokenizer`.

In [17]:
notebook.tqdm.pandas()

tokenized = data.loc[:, 'text_clear'].progress_apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=150))

Определим максимальную длину строки, заполним пропуски нулями и создадим маску. Все это требуется для работы BERT.

In [18]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

print('Max length:', max_len)

Max length: 150


Построим эмбеддинги для текста, которые после этого объединим в признаки, которая будет использовать модель при обучении. Эмбеддинги будем строить батчами.

In [19]:
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size + 1)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

features = np.concatenate(embeddings)

Разделим данные на обучающую и тестовую выборки.

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    features, data['toxic'], test_size=0.2, random_state=404)

print('Размер обучающей выборки:', X_train.shape)
print('Размер тестовой выборки:', X_test.shape)

Размер обучающей выборки: (127656, 768)
Размер тестовой выборки: (31915, 768)


Обучим модель линейной регрессии.

In [24]:
%%time

model_lr = LogisticRegression(solver='sag', max_iter=1000)
model_lr.fit(X_train, y_train)
y_pred = model_lr.predict(X_test)

print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))
print('F1 score: {:.3f}'.format(f1_score(y_test, y_pred)))

Accuracy: 0.950
F1 score: 0.725
CPU times: user 12min 46s, sys: 343 ms, total: 12min 47s
Wall time: 12min 46s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## Вывод 2.

Значение F1 при использовании `BertModel` получилось **F1 = 0.725**, что практически совпадает с метрикой при использовании преобразования TF-IDF. Для улучшения качества предсказаний можно попробовать воспользоваться классификатором `BertForSequenceClassification`.